<a href="https://colab.research.google.com/github/BRIJNANDA1979/DrivenDATA_competetion/blob/main/Version_1_3_CNN_for_DRIVEDATA_project_accuracy_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**bold text** After Finishing Step 2 of preperation of Data....Now its time to run CNN on newly prepared data in Project 2023 folder under name
"TRAIN_DATA_for_CNN

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec 15 02:37:34 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 13 03:02:04 2021

@author: BRIJB
"""

from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import the needed packages
import tensorflow as tf
from keras import losses
from keras import optimizers
from keras import metrics
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow.keras as keras
from keras.preprocessing import image
import numpy as np
from keras.models import model_from_json
import os





In [4]:
# define and move to dataset directory
datasetdir = "/content/drive/MyDrive/Project 2023/DRIVEN_DATA competetion/TRAIN_DATA_for_CNN"
import os
os.chdir(datasetdir)

In [5]:
# shortcut to the ImageDataGenerator class
ImageDataGenerator = keras.preprocessing.image.ImageDataGenerator
gen = ImageDataGenerator()
iterator = gen.flow_from_directory(
    os.getcwd(),
    target_size=(512,512),
    #classes=('Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential','SeaLake')
    classes = ('antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent')

)

Found 16488 images belonging to 8 classes.


In [6]:
batch = iterator.next()
print(len(batch))
print(type(batch[0]))
print(batch[0].shape)
print(batch[0].dtype)
#the first element is an array of 32 images with 64X64 pixels, and 3 color channels, encoded as floats in the range 0 to 255
#The second element contains the 32 corresponding labels.

2
<class 'numpy.ndarray'>
(32, 512, 512, 3)
float32


In [7]:
batch = iterator.next()
print(len(batch))
print(type(batch[1]))
print(batch[1].shape)
print(batch[1].dtype)

2
<class 'numpy.ndarray'>
(32, 8)
float32


In [8]:
imgdatagen = ImageDataGenerator(
    rescale = 1/255.,
    validation_split = 0.2,
)

batch_size = 32
height, width = (512,512)
train_dataset = imgdatagen.flow_from_directory(
    os.getcwd(),
    target_size = (height, width),
    #classes = ('Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential','SeaLake'),
    classes = ('antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent'),
    batch_size = batch_size,
    subset = 'training'
)

val_dataset = imgdatagen.flow_from_directory(
    os.getcwd(),
    target_size = (height, width),
    #classes = ('Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential','SeaLake'),
    classes = ('antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent'),
    batch_size = batch_size,
    subset = 'validation'
)
#In this case out of 5000 images: Training: 4000 Validation: 20% of 4000 i.e. 1000
model = keras.models.Sequential()

Found 13195 images belonging to 8 classes.
Found 3293 images belonging to 8 classes.


In [9]:

initializers = {

}

model.add(
    keras.layers.Conv2D(
        24, 5, input_shape=(512,512,3),
        activation='relu',
    )
)
model.add( keras.layers.MaxPooling2D(2) )
model.add(
    keras.layers.Conv2D(
        48, 5, activation='relu',
    )
)
model.add( keras.layers.MaxPooling2D(2) )
model.add(
    keras.layers.Conv2D(
        96, 5, activation='relu',
    )
)
model.add( keras.layers.Flatten() )
model.add( keras.layers.Dropout(0.4) )

model.add( keras.layers.Dense(
    8, activation='softmax',
    )
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 508, 508, 24)      1824      
                                                                 
 max_pooling2d (MaxPooling2  (None, 254, 254, 24)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 250, 250, 48)      28848     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 125, 125, 48)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 121, 121, 96)      115296    
                                                                 
 flatten (Flatten)           (None, 1405536)           0

In [10]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adamax(lr=0.001),
              metrics=['acc'])

In [11]:
history = model.fit_generator(
    train_dataset,
    validation_data = val_dataset,
    workers=1,
    epochs=1,
)

#result of this model is not impressive, but can be imroved
#443/443 [==============================] - 363s 802ms/step - loss: 0.3290 - acc: 0.3072 - val_loss: 0.2291 - val_acc: 0.5466

<ipython-input-11-78c51bf6350c>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


413/413 [==============================] - 4439s 11s/step - loss: 0.3117 - acc: 0.5142 - val_loss: 0.1969 - val_acc: 0.6951


Generate Test Data

In [12]:
category = ['1', '2', '3', '4', '5', '6', '7', '8']

In [13]:
test_dir = '/content/drive/MyDrive/TEST_512_sized_images'

In [14]:
desired_batch_size=32

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(512,512),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=desired_batch_size)

filenames = test_generator.filenames
nb_samples = len(filenames)/desired_batch_size



Found 188 images belonging to 8 classes.


In [ ]:

predict = model.predict_generator(test_generator,steps =
                                   int(nb_samples))

<ipython-input-15-12448886d6e0>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict = model.predict_generator(test_generator,steps =


In [ ]:
# result 1 with Epochs =10   Epoch 10/10

#86/86 [==============================] - 45s 518ms/step - loss: 0.1856 - acc: 0.7538 - val_loss: 0.1883 - val_acc: 0.7555